# Analyser des données non structurées : du texte

In [1]:
import pandas as pd

## Charger les données

In [2]:
url = "https://zenodo.org/record/5827206/files/SOSP_Export_base%20de%20donn%C3%A9es%20diffusable.csv"
data = pd.read_csv(url)

## Utiliser des outils déjà disponibles

Evidement ces étapes sont souvent faites et il est possible d'accélérer

In [1]:
# à ré-exécuter si besoin (long)
# %pip install -U spacy

In [2]:
# à ré-exécuter si besoin (long)
# !python -m spacy download fr_dep_news_trf

Par exemple sur les stopwords

In [27]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [3]:
#fr_stop

Mais aussi pour créer des sacs de mots

Il y a des outils de haut niveau paramétrables pour directement obtenir les formats souhaités. Mais cela demande potentiellement de rentrer un peu dans la bibliothèque.

scikit-learn a un format générique :
* on crée un objet,
* on le `fit` sur un ensemble ou sous-ensemble de données (eg. d'entraînement),
* on l'utilise pour transformer les données (`transform`) ou pour faire une prédiction (`predict`)

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
vectorizer = CountVectorizer(stop_words=list(fr_stop), min_df=5)

In [30]:
X = vectorizer.fit_transform(data["outils"])

/home/mathieu/miniconda3/envs/pyshs/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


In [31]:
X.toarray().shape

(1089, 260)

In [6]:
#vectorizer.get_feature_names_out()

D'autres traitements sont directement disponibles, par exemple le calcul de tf-idf.

# Utiliser les modèles de langage

## Quelques mots sur le principe des transformers

## Présentation de la bibliothèque SpaCy

## Application sur le champ libre

- récupérer les noms ?

In [54]:
questions = pd.read_csv("https://zenodo.org/record/5827206/files/SOSP_metadonnees_variables.csv", header=31)
questions

,Nom variable,Question,Modalités de réponse proposées,Colonne,Observations _Commentaires
0,ID,NaN,NaN,A,Identifiant de l’observé
1,contexte_travail,"Habituellement, pour une opération de recherch...",je travaille seul nous sommes entre 2 et 5 p...,B,NaN
2,fonction_recherche,Quelle est votre fonction dans la recherche ?,doctorant / doctorante (y compris CIFRE) maî...,C,NaN
3,statut_professionnel,Quel est votre statut professionnel ?,Fonctionnaire ou assimilé en CDD en CDI po...,D,NaN
4,statut_professionnel_autre,"Si, autre pouvez-vous préciser ?",Question ouverte,E,NaN
...,...,...,...,...,...
126,compatibilite_diffusion_partage\n,34 - Le partage et la diffusion des données vo...,Question ouverte,DW,mots anonymisés sous forme de XXXXX
127,evolutions_pratiques_numeriques,"35 - Selon vous, vos pratiques numériques vont...",Question ouverte,DX,mots anonymisés sous forme de XXXXX
128,sexe,Quel est votre genre ?,Une femme Un homme Je ne souhaite pas répondre,DY,NaN
129,Disciplines_9niv,Quelle est votre discipline principale de rech...,Sciences de l'ingénieur Sciences humaines\nSc...,DZ,recodage en 9 disciplines


In [58]:
questions[questions["Modalités de réponse proposées"] == "Question ouverte"]

,Nom variable,Question,Modalités de réponse proposées,Colonne,Observations _Commentaires
4,statut_professionnel_autre,"Si, autre pouvez-vous préciser ?",Question ouverte,E,NaN
43,volumetrie_donnees,Quelle est la quantité estimée de ces données ...,Question ouverte,AR,NaN
46,Tiers_archivage,"Si oui, à quels tiers les confiez-vous ?",Question ouverte,AU,NaN
47,Tiers_archivage_recod,"Si oui, à quels tiers les confiez-vous ?",Question ouverte,AV,Recodage des réponses de la variable précédente
57,logiciel_production_donnees,Quels logiciels utilisez-vous pour produire de...,Question ouverte,BF,NaN
60,outils_nettoyage,Pouvez-vous précisez lesquels ?,Question ouverte,BI,NaN
62,Outils_analyses,Pouvez-vous précisez lesquels ?,Question ouverte,BK,NaN
64,Outils_visualisation,Pouvez-vous précisez lesquels ?,Question ouverte,BM,NaN
75,outils_production_figure,Quels outils utilisez-vous pour générer des fi...,Question ouverte,BX,NaN
126,compatibilite_diffusion_partage\n,34 - Le partage et la diffusion des données vo...,Question ouverte,DW,mots anonymisés sous forme de XXXXX


In [76]:
questions_ouvertes = (
    questions[questions["Modalités de réponse proposées"] == "Question ouverte"]["Nom variable"]
    .str.lower()
    .str.strip()
    .tolist()
)
questions_ouvertes

['statut_professionnel_autre',
 'volumetrie_donnees',
 'tiers_archivage',
 'tiers_archivage_recod',
 'logiciel_production_donnees',
 'outils_nettoyage',
 'outils_analyses',
 'outils_visualisation',
 'outils_production_figure',
 'compatibilite_diffusion_partage',
 'evolutions_pratiques_numeriques']

In [79]:
data = data.rename(columns={x: x.lower().strip() for x in data.columns.tolist()})
data

,id,contexte_travail,fonction_recherche,statut_professionnel,statut_professionnel_autre,annee_premiere_publi,systeme_exploitation,usage_telephone_mobile,outils_gestion_travail_av_confinement_visioconférence,outils_gestion_travail_av_confinement_planification_rdv,...,souhait_partage_donnees_produites_sans_restriction,compatibilite_diffusion_partage,evolutions_pratiques_numeriques,sexe,disciplines_9niv,annee_de_naissance-recod10niv,outils,outils_alt,outils_token,labels
0,1,nous sommes entre 6 et 10 personnes,professeur / professeure des universités et as...,fonctionnaire ou assimilé,NaN,1995,Windows,rarement,parfois,souvent,...,oui,oui,je ne sais pas,un homme,Sciences humaines,61 à 65 ans,lime survey tableur pour les données statistiques,lime survey tableur pour les données statistiques,"[lime, survey, tableur, statistiques]",2
1,2,nous sommes plus de 50 personnes,ingénieur / ingénieure de recherche,en CDI,NaN,2005,Windows,rarement,parfois,parfois,...,oui,NaN,NaN,un homme,Médecine,46 à 50 ans,excell word clean outil statistique,excell word clean outil statistique,"[excell, word, clean, outil, statistique]",0
2,3,nous sommes entre 2 et 5 personnes,maître / maîtresse de conférences et assimilés,fonctionnaire ou assimilé,NaN,2006,Windows,parfois,rarement,souvent,...,oui,"oui, tout est dans la chronologie",non,un homme,Médecine,36 à 40 ans,jasp,jasp,[jasp],3
3,4,nous sommes entre 2 et 5 personnes,"directeur / directrice de recherche (CNRS, INS...",fonctionnaire ou assimilé,NaN,1989,Windows;autres OS (Android...),souvent,rarement,souvent,...,non,Oui,Aucune idée,une femme,Médecine,56 à 60 ans,"question peu claire. excel, spss, logiciels d...","question peu claire. excel, spss, logiciels d...","[question, peu, claire, excel, spss, imagerie,...",1
4,5,nous sommes entre 2 et 5 personnes,doctorant / doctorante (y compris CIFRE),en CDD,NaN,2015,MacOS,souvent,rarement,parfois,...,oui,Oui,Oui,une femme,Médecine,31 à 35 ans,? excel r image j dti studii,? excel r image j dti studii,"[excel, r, image, j, dti, studii]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,1085,nous sommes entre 2 et 5 personnes,"chargé / chargée de recherche (CNRS, INSERM, I...",en CDI,NaN,2004,Windows,parfois,jamais,parfois,...,oui,"Oui, si les innovations ont été protégées en a...",Mise en place d'un plan de partage des données...,un homme,NaN,41 à 45 ans,epiinfo\nxlstat epiinfo\nxlstat\nexcel,epiinfo\nxlstat epiinfo\nxlstat\nexcel,"[epiinfo, xlstat, epiinfo, xlstat, excel]",1
1085,1086,nous sommes entre 2 et 5 personnes,"chargé / chargée de recherche (CNRS, INSERM, I...",fonctionnaire ou assimilé,NaN,1987,Linux,jamais,rarement,rarement,...,oui,je ne pratique pas des 'activités de valorisat...,je vais prendre ma retraite,un homme,NaN,61 à 65 ans,"xpdf, evince","xpdf, evince","[xpdf, evince]",1
1086,1087,nous sommes entre 2 et 5 personnes,maître / maîtresse de conférences et assimilés,fonctionnaire ou assimilé,NaN,1992,Windows,jamais,souvent,souvent,...,oui,oui,NaN,un homme,NaN,56 à 60 ans,"openrefine, talend","openrefine, talend","[openrefine, talend]",0
1087,1088,nous sommes entre 2 et 5 personnes,"directeur / directrice de recherche (CNRS, INS...",fonctionnaire ou assimilé,NaN,1982,Linux,jamais,rarement,souvent,...,non,Je ne sais pas vraiment repondre a ces questio...,Je ne sais pas,une femme,NaN,61 à 65 ans,,,[],2


In [82]:
data[questions_ouvertes].apply(lambda x: x.str.len().describe(), axis=0).transpose()

,count,mean,std,min,25%,50%,75%,max
statut_professionnel_autre,7.0,49.000000,41.717303,21.0,24.00,32.0,51.50,139.0
volumetrie_donnees,1081.0,21.663275,33.037989,1.0,4.00,11.0,25.00,379.0
tiers_archivage,505.0,23.154455,24.143747,1.0,9.00,18.0,28.00,255.0
tiers_archivage_recod,491.0,18.517312,11.105487,3.0,11.00,13.0,24.00,61.0
logiciel_production_donnees,1082.0,39.624769,52.660816,1.0,12.00,25.0,46.00,619.0
outils_nettoyage,333.0,22.408408,29.819460,1.0,6.00,13.0,26.00,263.0
outils_analyses,646.0,26.287926,37.446446,1.0,7.00,15.0,31.75,413.0
outils_visualisation,743.0,23.269179,29.747549,1.0,7.00,15.0,27.50,316.0
outils_production_figure,852.0,22.438967,26.127542,1.0,8.75,17.0,28.00,375.0
compatibilite_diffusion_partage,968.0,55.552686,123.046083,1.0,3.00,4.0,59.25,1459.0


In [84]:
epn = data["evolutions_pratiques_numeriques"].dropna()
epn

0                                          je ne sais pas
2                                                     non
3                                             Aucune idée
4                                                     Oui
6       Je ne sais pas. Fonction de l'accès à des form...
                              ...                        
1083    Les pratiques vont se renforcer et se professi...
1084    Mise en place d'un plan de partage des données...
1085                          je vais prendre ma retraite
1087                                       Je ne sais pas
1088    Oui, vers plus d'ouverture mais je ne sais pas...
Name: evolutions_pratiques_numeriques, Length: 882, dtype: object

In [85]:
import spacy

In [ ]:
nlp = spacy.load("fr_dep_news_trf")

In [101]:
un_texte = "La petite ferme la porte. La belle brise la glace. Le chat porte un chapeau."
doc = nlp(un_texte)
doc

La petite ferme la porte. La belle brise la glace. Le chat porte un chapeau.

In [109]:
for token in doc:
    print("\t".join(str(x) for x in [token.text, token.lemma_, token.pos_, token.morph, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop]))

La	le	DET	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	DET	det	Xx	True	True
petite	petit	ADJ	Gender=Fem|Number=Sing	ADJ	amod	xxxx	True	False
ferme	ferme	NOUN	Gender=Fem|Number=Sing	NOUN	nsubj	xxxx	True	False
la	le	DET	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	DET	det	xx	True	True
porte	porte	NOUN	Gender=Fem|Number=Sing	NOUN	ROOT	xxxx	True	False
.	.	PUNCT		PUNCT	punct	.	False	False
La	le	DET	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	DET	det	Xx	True	True
belle	bel	ADJ	Gender=Fem|Number=Sing	ADJ	amod	xxxx	True	False
brise	brise	NOUN	Gender=Fem|Number=Sing	NOUN	ROOT	xxxx	True	False
la	le	DET	Definite=Def|Gender=Fem|Number=Sing|PronType=Art	DET	det	xx	True	True
glace	glace	NOUN	Gender=Fem|Number=Sing	NOUN	nmod	xxxx	True	False
.	.	PUNCT		PUNCT	punct	.	False	False
Le	le	DET	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	DET	det	Xx	True	True
chat	chat	NOUN	Gender=Masc|Number=Sing	NOUN	nsubj	xxxx	True	False
porte	porte	VERB	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=

In [103]:
from spacy import displacy

In [104]:
displacy.render(doc, style="dep")

In [105]:
displacy.render(list(doc.sents), style="dep")

In [106]:
displacy.render(doc, style="ent")

/home/mathieu/miniconda3/envs/pyshs/lib/python3.11/site-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [110]:
un_autre_texte = "Jeanne part pour Genève. Elle poursuivra ensuite son voyage vers Big Apple."
doc2 = nlp(un_autre_texte)

In [111]:
displacy.render(doc2, style="ent")

/home/mathieu/miniconda3/envs/pyshs/lib/python3.11/site-packages/spacy/displacy/__init__.py:215: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [4]:
#!python -m spacy download fr_core_news_md

In [114]:
nlp_md = spacy.load("fr_core_news_md")

In [115]:
doc2_md = nlp_md(un_autre_texte)
doc2_md

Jeanne part pour Genève. Elle poursuivra ensuite son voyage vers Big Apple.

In [117]:
displacy.render(doc2_md, style="ent")

In [ ]:
epns = epn.tolist()
epns

In [ ]:
docs_md = [nlp_md(x) for x in epns]
docs_md

In [ ]:
displacy.render(docs_md, style="ent")

# Petite application de haut niveau : sentiment analysis

ATTENTION CA MARCHE PAS SUR MON MAC : MATTHIEU ?


Identifier les personnes qui ont un ressenti négatif par rapport aux évolutions du numériques

https://huggingface.co/cmarkea/distilcamembert-base-sentiment

In [123]:
from transformers import pipeline

analyzer = pipeline(
    task='text-classification',
    model="cmarkea/distilcamembert-base-sentiment",
    tokenizer="cmarkea/distilcamembert-base-sentiment"
)

In [ ]:
epsn_sent = analyzer(epns)
epsn_sent

In [ ]:
for reponse, sent in zip(epn, epsn_sent):
    print(reponse, sent["label"], sent["score"])

In [ ]:
for reponse, sent in zip(epn, epsn_sent):
    if sent["label"] in ["1 star", "2 stars"]:
        print(reponse, sent["label"], sent["score"])

In [129]:
df_sent = pd.DataFrame(
    data={
        "reponse": epn,
        "sentiment": [x["label"] for x in epsn_sent],
        "sent_score": [x["score"] for x in epsn_sent],
    }
)
df_sent

,reponse,sentiment,sent_score
0,je ne sais pas,3 stars,0.311197
2,non,1 star,0.285603
3,Aucune idée,1 star,0.592063
4,Oui,4 stars,0.374047
6,Je ne sais pas. Fonction de l'accès à des form...,3 stars,0.314710
...,...,...,...
1083,Les pratiques vont se renforcer et se professi...,4 stars,0.468114
1084,Mise en place d'un plan de partage des données...,4 stars,0.474919
1085,je vais prendre ma retraite,5 stars,0.342706
1087,Je ne sais pas,3 stars,0.307354


In [137]:
df_sent[(df_sent["sentiment"] == "1 star") | (df_sent["sentiment"] == "2 stars")]

,reponse,sentiment,sent_score
2,non,1 star,0.285603
3,Aucune idée,1 star,0.592063
24,Aucune idée,1 star,0.592063
25,Non,1 star,0.237107
26,"NON,je suis trop vieux",1 star,0.725573
...,...,...,...
1058,Il est possible que je sois obligé d'utiliser ...,2 stars,0.433645
1061,non,1 star,0.285603
1070,Difficile à dire. En l'absence de mise à dispo...,2 stars,0.515802
1071,non elles évolueront peu. L'objectif étant dés...,2 stars,0.305642


In [139]:
df_sent[(df_sent["sentiment"] == "1 star")].sort_values(by="sent_score", ascending=False)

,reponse,sentiment,sent_score
1055,"augmenter nos emmerdes. On est bouffé par 150,...",1 star,0.930310
26,"NON,je suis trop vieux",1 star,0.725573
159,Non car aucune formation et aucun temps pour l...,1 star,0.687645
566,aucune idée...,1 star,0.661838
58,Aucune idée.,1 star,0.658081
...,...,...,...
709,Non,1 star,0.237107
891,Non,1 star,0.237107
543,Non,1 star,0.237107
967,Incrémentale,1 star,0.226097


In [142]:
df_sent[(df_sent["sentiment"] == "1 star")].sort_values(by="sent_score", ascending=False).iloc[0]["reponse"]

"augmenter nos emmerdes. On est bouffé par 150,000 trucs à la con, et on nous en rajoute ... \nLes équipes sont à bout, les crédits n'existent plus, on a 0 reconnaissance.\nfaut-il répondre à toutes ces demandes diverses et avariées, réponse: comme pour le COVID: négatif."